In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from tabulate import tabulate
import pandas as pd

# Define the quantization matrix
quantization_matrix = np.array([
    [16, 11, 10, 16, 24, 40, 51, 61],
    [12, 12, 14, 19, 26, 58, 60, 55],
    [14, 13, 16, 24, 40, 57, 69, 56],
    [14, 17, 22, 29, 51, 87, 80, 62],
    [18, 22, 37, 56, 68, 109, 103, 77],
    [24, 35, 55, 64, 81, 104, 113, 92],
    [49, 64, 78, 87, 103, 121, 120, 101],
    [72, 92, 95, 98, 112, 100, 103, 99]
], dtype=np.float32)

def zigzag_scan(block: np.ndarray[np.float32]) -> np.ndarray[np.float32]:
    """Scans a block in zigzag order and return a 1D array."""
    block_size = block.shape[0]
    zigzag_arr = np.concatenate([
        np.diagonal(block[::-1, :], i)[::(2 * (i % 2) - 1)]
        for i in range(1 - block_size, block_size)
    ])
    return zigzag_arr

def zigzag_unscan(zigzag_arr: np.ndarray[np.float32], block_size: int) -> np.ndarray[np.float32]:
    """Unscans a 1D array in zigzag order and return a 2D array."""
    block = np.zeros((block_size, block_size), dtype=np.float32)
    x, y = 0, 0
    for num in zigzag_arr:
        block[x, y] = num
        if (x + y) % 2 == 0:
            if y == block_size - 1:
                x += 1
            elif x == 0:
                y += 1
            else:
                x -= 1
                y += 1
        else:
            if x == block_size - 1:
                y += 1
            elif y == 0:
                x += 1
            else:
                x += 1
                y -= 1
    return block

def jpeg_compression(image: np.ndarray[np.uint8], block_size=8, num_coeffs=10) -> np.ndarray[np.uint8]:
    """Compresses a grayscale image using JPEG-like compression."""
    height, width = image.shape
    h = height // block_size * block_size
    w = width // block_size * block_size
    image = image[:h, :w]
    compressed = np.zeros_like(image, dtype=float)
    for i in range(0, h, block_size):
        for j in range(0, w, block_size):
            block = image[i:i + block_size, j:j + block_size].astype(np.float32) - 128
            dct_block = cv2.dct(block)
            quantized = np.round(dct_block / quantization_matrix)
            zigzag_quantized = zigzag_scan(quantized)
            zigzag_quantized = zigzag_quantized[:num_coeffs]
            dequantized = zigzag_unscan(np.pad(zigzag_quantized, (0, block_size**2 - num_coeffs)), block_size)
            dequantized *= quantization_matrix
            idct_block = cv2.idct(dequantized)
            compressed[i:i + block_size, j:j + block_size] = idct_block + 128
    return np.clip(compressed, 0, 255).astype(np.uint8)

def rgb2ycbcr(im: np.ndarray[np.uint8]) -> np.ndarray[np.uint8]:
    """Convert RGB image to YCbCr color space."""
    xform = np.array([[.299, .587, .114], [-.1687, -.3313, .5], [.5, -.4187, -.0813]])
    ycbcr = im.astype(float).dot(xform.T)
    ycbcr[:, :, [1, 2]] += 128
    return np.uint8(ycbcr)

def ycbcr2rgb(im: np.ndarray[np.uint8]) -> np.ndarray[np.uint8]:
    """Convert YCbCr image back to RGB color space."""
    xform = np.array([[1, 0, 1.402], [1, -0.34414, -.71414], [1, 1.772, 0]])
    rgb = im.astype(float)
    rgb[:, :, [1, 2]] -= 128
    rgb = rgb.dot(xform.T)
    np.clip(rgb, 0, 255, out=rgb)
    return np.uint8(rgb)

def calculate_compression_ratio(original: np.ndarray[np.uint8], num_coeffs: int, block_size=8) -> float:
    """Calculate the JPEG compression ratio."""
    total_coeffs = original.size
    retained_coeffs = num_coeffs * num_coeffs * (total_coeffs // (block_size * block_size))
    return total_coeffs / retained_coeffs

def psnr(true: np.ndarray[np.uint8], test: np.ndarray[np.uint8]) -> float:
    """Calculate the PSNR between two images."""
    mse = np.mean((true.astype(float) - test.astype(float)) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * np.log10(255.0) - 10 * np.log10(mse)

def plot_results(original: np.ndarray[np.uint8], compressed: np.ndarray[np.uint8], title: str) -> None:
    """Plot the original and compressed images side by side."""
    plt.figure(figsize=(12, 6))
    plt.subplot(121)
    plt.imshow(original, cmap='gray' if len(original.shape) == 2 else None)
    plt.title('Original')
    plt.axis('off')

    plt.subplot(122)
    plt.imshow(compressed, cmap='gray' if len(compressed.shape) == 2 else None)
    plt.title(f'Compressed\n{title}')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

def process_image(image_path: str, num_coeffs: int = 10) -> dict:
    """Process a single image and return compression details."""
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    compressed_gray = jpeg_compression(img_gray, num_coeffs=num_coeffs)
    psnr_gray = psnr(img_gray, compressed_gray)
    cr_gray = calculate_compression_ratio(img_gray, num_coeffs)

    img_ycbcr = rgb2ycbcr(img_rgb)
    compressed_y = jpeg_compression(img_ycbcr[:, :, 0], num_coeffs=num_coeffs)
    compressed_cb = jpeg_compression(img_ycbcr[:, :, 1], num_coeffs=num_coeffs)
    compressed_cr = jpeg_compression(img_ycbcr[:, :, 2], num_coeffs=num_coeffs)
    compressed_ycbcr = np.dstack((compressed_y, compressed_cb, compressed_cr))
    compressed_rgb = ycbcr2rgb(compressed_ycbcr)

    psnr_color = psnr(img_rgb, compressed_rgb)
    cr_color = calculate_compression_ratio(img_rgb, num_coeffs)

    plot_results(img_gray, compressed_gray, f'PSNR: {psnr_gray:.2f}, CR: {cr_gray:.2f}')
    plot_results(img_rgb, compressed_rgb, f'PSNR: {psnr_color:.2f}, CR: {cr_color:.2f}')

    return {
        'Image': os.path.basename(image_path),
        'Num Coefficients': num_coeffs,
        'PSNR Gray': psnr_gray,
        'CR Gray': cr_gray,
        'PSNR Color': psnr_color,
        'CR Color': cr_color
    }

def process_images_in_directory(directory: str, num_coeffs_list: list[int]) -> None:
    """Process all images in a directory and output results."""
    results = []
    for filename in os.listdir(directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            image_path = os.path.join(directory, filename)
            for num_coeffs in num_coeffs_list:
                print(f'Processing {filename} with {num_coeffs} coefficients...')
                result = process_image(image_path, num_coeffs=num_coeffs)
                results.append(result)

    df = pd.DataFrame(results)
    print(tabulate(df, headers='keys', tablefmt='pretty'))

# Example usage
if __name__ == "__main__":
    directory = '/Users/annsb/Desktop/VC Project/Image-Compression-using-MDCT/images/colored' #Add path to the images folder
    num_coeffs_list = [5, 10, 20]  # Example number of coefficients
    process_images_in_directory(directory, num_coeffs_list)
